In [31]:
## Introducing GINI as stopping criteria
# Define function to calculate acumlative frequencies
import math
def acum(array):
  # Take in mind that the result not is order
  new, N = [], sum(array)
  acum = 0
  for i in array:
    acum = acum + i
    new.append(acum/N)
  return new

#define function to calculate Gini coefficient
def gini(data):
  Y = acum(sorted(data))
  N = acum([1 for _ in range(len(data))])
  area = 0
  for index in range(1,len(data)):
    area +=  ((Y[index-1] + Y[index])) *  (N[index] - N[index-1])
  return 1 - area

## theil give us a more stable computation!
def theil(array):
  Y = sum(array)
  N = len(array)
  relative = [y/Y for y in array]
  Theil = sum([y * math.log(y * N) for y in relative])/math.log(N)
  return Theil

In [32]:
import pandas as pd
import geopandas as gpd

In [33]:
url  = "https://raw.githubusercontent.com/it-ces/Datasets/main/icfes.csv"

In [34]:
df = pd.read_csv(url)

In [35]:
df.columns

Index(['Unnamed: 0', 'ESTU_TIPODOCUMENTO', 'ESTU_NACIONALIDAD', 'ESTU_GENERO',
       'ESTU_FECHANACIMIENTO', 'PERIODO', 'ESTU_CONSECUTIVO',
       'ESTU_ESTUDIANTE', 'ESTU_PAIS_RESIDE', 'ESTU_TIENEETNIA',
       'ESTU_DEPTO_RESIDE', 'ESTU_COD_RESIDE_DEPTO', 'ESTU_MCPIO_RESIDE',
       'ESTU_COD_RESIDE_MCPIO', 'FAMI_EDUCACIONPADRE', 'FAMI_EDUCACIONMADRE',
       'FAMI_ESTRATOVIVIENDA', 'FAMI_PERSONASHOGAR', 'FAMI_CUARTOSHOGAR',
       'FAMI_TIENEINTERNET', 'FAMI_TIENECOMPUTADOR', 'FAMI_TIENELAVADORA',
       'FAMI_TIENEHORNOMICROOGAS', 'FAMI_TIENESERVICIOTV',
       'FAMI_TIENEAUTOMOVIL', 'FAMI_TIENEMOTOCICLETA',
       'FAMI_TIENECONSOLAVIDEOJUEGOS', 'FAMI_NUMLIBROS',
       'ESTU_DEDICACIONLECTURADIARIA', 'ESTU_DEDICACIONINTERNET',
       'ESTU_HORASSEMANATRABAJA', 'ESTU_TIPOREMUNERACION',
       'FAMI_COMELECHEDERIVADOS', 'FAMI_COMECARNEPESCADOHUEVO',
       'FAMI_COMECEREALFRUTOSLEGUMBRE', 'FAMI_TRABAJOLABORPADRE',
       'FAMI_TRABAJOLABORMADRE', 'FAMI_SITUACIONECONOMICA',
       

In [36]:
df['PUNT_MATEMATICAS'].std()/df['PUNT_MATEMATICAS'].mean()

0.24245793722133713

In [37]:
gini(df['PUNT_MATEMATICAS'])

0.13761161587415416

In [38]:
theil(df['PUNT_MATEMATICAS'])

0.0038415204519814724

In [39]:
df['PUNT_MATEMATICAS'].describe()

count    2600.000000
mean       50.524615
std        12.250094
min        16.000000
25%        42.000000
50%        51.000000
75%        59.000000
max       100.000000
Name: PUNT_MATEMATICAS, dtype: float64

In [40]:
df.columns

Index(['Unnamed: 0', 'ESTU_TIPODOCUMENTO', 'ESTU_NACIONALIDAD', 'ESTU_GENERO',
       'ESTU_FECHANACIMIENTO', 'PERIODO', 'ESTU_CONSECUTIVO',
       'ESTU_ESTUDIANTE', 'ESTU_PAIS_RESIDE', 'ESTU_TIENEETNIA',
       'ESTU_DEPTO_RESIDE', 'ESTU_COD_RESIDE_DEPTO', 'ESTU_MCPIO_RESIDE',
       'ESTU_COD_RESIDE_MCPIO', 'FAMI_EDUCACIONPADRE', 'FAMI_EDUCACIONMADRE',
       'FAMI_ESTRATOVIVIENDA', 'FAMI_PERSONASHOGAR', 'FAMI_CUARTOSHOGAR',
       'FAMI_TIENEINTERNET', 'FAMI_TIENECOMPUTADOR', 'FAMI_TIENELAVADORA',
       'FAMI_TIENEHORNOMICROOGAS', 'FAMI_TIENESERVICIOTV',
       'FAMI_TIENEAUTOMOVIL', 'FAMI_TIENEMOTOCICLETA',
       'FAMI_TIENECONSOLAVIDEOJUEGOS', 'FAMI_NUMLIBROS',
       'ESTU_DEDICACIONLECTURADIARIA', 'ESTU_DEDICACIONINTERNET',
       'ESTU_HORASSEMANATRABAJA', 'ESTU_TIPOREMUNERACION',
       'FAMI_COMELECHEDERIVADOS', 'FAMI_COMECARNEPESCADOHUEVO',
       'FAMI_COMECEREALFRUTOSLEGUMBRE', 'FAMI_TRABAJOLABORPADRE',
       'FAMI_TRABAJOLABORMADRE', 'FAMI_SITUACIONECONOMICA',
       

In [41]:
#'COLE_COD_DEPTO_UBICACION','COLE_DEPTO_UBICACION'
df['mean-dpto'] = df.groupby(by=['COLE_COD_DEPTO_UBICACION'])['PUNT_MATEMATICAS'].transform("mean")

In [42]:
df = df[['mean-dpto', 'COLE_COD_DEPTO_UBICACION', 'COLE_DEPTO_UBICACION' ]]

In [43]:
df = df.drop_duplicates(subset=['COLE_DEPTO_UBICACION']).reset_index(drop=True)

In [44]:
df['COLE_COD_DEPTO_UBICACION']  = df['COLE_COD_DEPTO_UBICACION'].astype('int64')

In [45]:
geomap  = gpd.read_file("MGN_DPTO_POLITICO.shp")
geomap['DPTO_CCDGO'] = geomap['DPTO_CCDGO'].astype('float64')
map_plot = geomap.merge(df, left_on=['DPTO_CCDGO'], right_on=[ 'COLE_COD_DEPTO_UBICACION'])

DriverError: MGN_DPTO_POLITICO.shp: No such file or directory

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
map_plot.replace(0,np.nan, inplace=True)
map_plot.plot(
column="mean-dpto",
cmap='binary',
edgecolor='black',
alpha=1,
legend=True,
figsize=(14, 14),
missing_kwds={
        "color": "white",
        "edgecolor": "gray",
        "hatch": "///",
        "label": "Missing values",    },
legend_kwds={
     'label': "Tasa promedio municipal de lectura crítica",
    'orientation': "vertical"
 })
plt.savefig("Mapa.png")